# Project: Herseninfarct
### Team: Undefined
### Teamleden:
- **Sebastiaan Westerlaken**
- **Michal Kakol**

In [1]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import iqr
from scipy.stats import shapiro
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from datetime import datetime

# Import src
from src.preprocessing import clean_data, normalize_data
from src.eda import eda_sum, impossible_values
from src.model import train_knn, train_lr, train_svm, train_decision_tree, train_random_forest, train_gradient_boosting, train_xgboost, train_custom_ensemble
from src.other_functions import evaluate_knn, export_submission

# Data
df_train = pd.read_csv("data/train.csv")
df_test = pd.read_csv("data/test.csv")
df_example = pd.read_csv("data/sample_submission.csv")
df_submission = pd.DataFrame(columns=["id", "stroke"])

- ChatGPT, 2025, prompt 1/2: Markdown Table Help. https://chatgpt.com/share/68e0c97c-687c-8002-b0f8-10e5e1e77ccb
| **Kolomnaam**                    | **Beschrijving**                                                                                |
| -------------------------------- | ----------------------------------------------------------------------------------------------- |
| `id`                             | Unieke identificatiecode voor elke persoon in de dataset.                                       |
| `age`                            | Leeftijd van de persoon (in jaren).                                                             |
| `hypertension`                   | Of de persoon een hoge bloeddruk (hypertensie) heeft (1 = Ja, 0 = Nee).                         |
| `heart_disease`                  | Of de persoon een hartaandoening heeft (1 = Ja, 0 = Nee).                                       |
| `avg_glucose_level`              | Gemiddeld glucosegehalte in het bloed (in mg/dL).                                               |
| `bmi`                            | Body Mass Index (BMI) — maat voor vetpercentage op basis van lengte en gewicht.                 |
| `gender_Female`                  | True als de persoon **vrouw** is; anders False.                                                 |
| `gender_Male`                    | True als de persoon **man** is; anders False.                                                   |
| `gender_Other`                   | True als de persoon een **ander gender** heeft; anders False.                                   |
| `ever_married_No`                | True als de persoon **nog nooit getrouwd** is; anders False.                                    |
| `ever_married_Yes`               | True als de persoon **ooit getrouwd** is; anders False.                                         |
| `work_type_Govt_job`             | True als de persoon een **overheidsbaan** heeft; anders False.                                  |
| `work_type_Never_worked`         | True als de persoon **nog nooit gewerkt** heeft; anders False.                                  |
| `work_type_Private`              | True als de persoon in de **private sector** werkt; anders False.                               |
| `work_type_Self-employed`        | True als de persoon **zelfstandig ondernemer** is; anders False.                                |
| `work_type_children`             | True als de persoon een **kind** is (nog geen onderdeel van de beroepsbevolking); anders False. |
| `Residence_type_Rural`           | True als de persoon in een **landelijk** gebied woont; anders False.                            |
| `Residence_type_Urban`           | True als de persoon in een **stedelijk** gebied woont; anders False.                            |
| `smoking_status_formerly smoked` | True als de persoon **vroeger rookte**; anders False.                                           |
| `smoking_status_never smoked`    | True als de persoon **nooit gerookt** heeft; anders False.                                      |
| `smoking_status_smokes`          | True als de persoon **momenteel rookt**; anders False.                                          |
| `stroke`                         | Doelvariabele — of de persoon ooit een **beroerte (stroke)** heeft gehad (1 = Ja, 0 = Nee).     |

(ChatGPT, 2025, prompt 1/2: Markdown Table Help)

# 1. Exploratieve Data Analyse<br>

Hieronder voeren wij eerst de exploratieve data analyse uit die wordt aangeroepen vanuit het src.eda bestand.

In [2]:
eda_sum(df_train)

NameError: name 'df_train' is not defined

## Exploratieve Data Analyse

### Beoordeling van de dataset

De dataset kan direct gebruikt worden met Scikit-Learn. Hij is ingeladen als een DataFrame, er zijn geen ontbrekende waarden en alle kolommen zijn numeriek. Het gaat om een combinatie van booleans, integers en floats.

Wat betreft de meetniveaus:
- Nominaal: heart_disease, hypertension, work_type_Self-employed, stroke
- Ordinaal: geen
- Interval: geen
- Ratio: age, avg_glucose_level, bmi

Hoewel sommige nominale variabelen als integers zijn opgeslagen, zijn ze binair van aard. Voor machine learning hoeft hier niets aan veranderd te worden.

### Correlaties en belangrijkste variabelen

Om te kijken naar mogelijke multicollineariteit en te zien welke features het meest relevant zijn voor de target (stroke), is een correlatie-analyse gedaan.

Sterkst gecorreleerde variabelen met stroke zijn:
- age 0.149
- heart_disease 0.105
- hypertension 0.084
- avg_glucose_level 0.077
- work_type_Self-employed 0.062
- smoking_status_formerly smoked 0.034
- bmi 0.021

Geen enkele correlatie is hoger dan 0.2, dus alle features kunnen worden behouden voor modelbouw.

### Belangrijkste observaties

Leeftijd blijkt duidelijk een belangrijke factor: jongere mensen hebben bijna geen kans op een beroerte, het risico neemt vooral toe na 35 jaar. BMI lijkt minder invloed te hebben dan verwacht. Voor roken valt op dat voormalige rokers iets meer risico lijken te hebben dan huidige rokers. Het gemiddeld glucosegehalte ligt hoger bij mensen met een beroerte, maar strokes komen voor bij een breed bereik aan waarden.

### Statistische kenmerken

Voor de niet-boolean variabelen zijn enkele kernstatistieken berekend. Age heeft een standaarddeviatie van 22.48 en is bijna symmetrisch verdeeld, met een licht afgeplatte verdeling. Avg_glucose_level ligt gemiddeld rond 104 met een standaarddeviatie van 42, de verdeling is rechts-scheef door enkele uitschieters. Deze uitschieters kunnen relevant zijn voor het model.

### Klasse-ongelijkheid

De dataset is sterk uit balans, met slechts 517 positieve stroke-gevallen. Daarom worden bij het evalueren van modellen vooral de F1-score en de confusion matrix gebruikt, omdat alleen accuracy niet genoeg zegt over de prestaties op de minderheidsklasse.

### Samenvatting

De dataset is geschikt voor machine learning. Er zijn geen missende waarden, geen sterke correlaties en de belangrijkste voorspellende variabelen lijken age, heart_disease, hypertension en avg_glucose_level. Vanwege de sterke class imbalance moet bij modeltraining extra aandacht worden besteed aan geschikte evaluatiemetrics.


## 1.1 Data opschonen en standaardiseren

In [3]:
# Opschonen
df_train_clean = clean_data(df_train)

# Features + target
X = df_train_clean.drop(columns=['stroke'])
y = df_train_clean['stroke']

# Standaardiseren en align train & test automatisch
X_scaled, X_scaled_test, scaler = normalize_data(X, df_test)

X_scaled.head()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,gender_Female,gender_Male,gender_Other,ever_married_No,...,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Rural,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,0.763423,-0.525188,-0.310285,-0.212086,-0.949341,0.631584,False,True,False,True,...,False,False,True,False,False,True,False,False,False,True
1,1.700336,1.475539,3.222846,-0.212086,2.652854,2.573031,True,False,False,False,...,False,False,False,True,False,True,False,False,True,False
2,-0.492650,-1.192097,-0.310285,-0.212086,0.752771,-1.257035,False,True,False,True,...,False,False,True,False,False,True,False,False,True,False
3,1.361419,0.230642,-0.310285,-0.212086,-0.918683,-0.041979,False,True,False,False,...,False,False,False,True,False,False,True,False,True,False
4,1.557020,0.319564,-0.310285,-0.212086,2.823255,0.037263,False,True,False,False,...,False,False,False,True,False,True,False,True,False,False


# 2. Evaluatie

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y)
evaluate_knn(X_train, X_test, y_train, y_test, n_neighbors=1)

[[6499   94]
 [  98    5]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      6593
           1       0.05      0.05      0.05       103

    accuracy                           0.97      6696
   macro avg       0.52      0.52      0.52      6696
weighted avg       0.97      0.97      0.97      6696



### Evaluatie van het model

Om het KNN-model te beoordelen gebruiken we enkele standaard metrics: accuracy, precision, recall en F1-score. Al deze waarden zijn af te leiden uit de confusion matrix, die laat zien hoe goed het model voorspellingen maakt.

### Confusion matrix

Voor het model met n_neighbors gelijk aan 1 ziet de confusion matrix er als volgt uit:

Actual zonder stroke voorspeld als geen stroke: 6499
Actual zonder stroke voorspeld als stroke: 94
Actual met stroke voorspeld als geen stroke: 98
Actual met stroke voorspeld als stroke: 5

In termen van de standaard terminologie betekent dit:
- True Negatives: 6499
- False Positives: 94
- False Negatives: 98
- True Positives: 5

### Uitleg van de metrics

Accuracy meet het aandeel correcte voorspellingen, in dit geval ongeveer 97 procent. Dit lijkt hoog, maar is misleidend door de scheve verdeling van de klassen.

Precision laat zien hoe vaak de voorspelde positieve gevallen echt positief zijn. Voor dit model is dat maar 0.05, dus het model voorspelt veel te vaak een stroke terwijl dat niet klopt.

Recall geeft aan hoeveel van de echte strokes het model herkent. Ook dit is 0.05, wat betekent dat bijna alle strokes gemist worden.

De F1-score combineert precision en recall tot één waarde die beide aspecten meeweegt. Bij scheve datasets geeft dit een realistischer beeld van het model.

### Waarom F1-score belangrijk is

De dataset bevat veel meer mensen zonder stroke dan met stroke. Bij zulke datasets kan een model dat altijd “geen stroke” voorspelt toch een hoge accuracy halen. Daarom is de F1-score hier een betere maatstaf. Het laat zien hoe goed het model de kleine positieve klasse oppikt en geeft een eerlijker beeld van de prestaties bij het detecteren van strokes.

# 3. Modelleren en uitleg modellen

## 3.1 KNN

### Model 1: K-Nearest Neighbors (KNN)

Hoe het model werkt:
KNN is een niet-parametrisch classificatiemodel dat een nieuwe observatie indeelt op basis van de afstand tot de dichtstbijzijnde trainingspunten. Meestal wordt de Euclidische afstand gebruikt, wat de rechte lijn tussen twee punten meet:

$$
d(p, q) = \sqrt{\sum_i (p_i - q_i)^2}
$$

Een alternatief is de Manhattan afstand, die de afstand langs de assen berekent:

$$
d(p, q) = \sum_i |p_i - q_i|
$$

Het model kiest de klasse van het nieuwe punt op basis van de meerderheid van de $k$ dichtstbijzijnde buren.

Waarom standaardisatie belangrijk is:
Omdat KNN afstanden vergelijkt tussen features, moeten alle features op dezelfde schaal staan. Zonder standaardisatie kunnen features met grotere waarden, zoals inkomen, de afstand te veel domineren vergeleken met kleinere waarden zoals leeftijd. Dit kan de voorspellingen van het model vervormen.

Beste hyperparameters:
Voor dit model zijn gekozen: metric = euclidean, k = 11 en weights = distance. Met deze instellingen behaalt het model een F1-score van 0.076.

Regularisatie bij KNN:
Regularisatie gebeurt hier indirect via de parameter $k$.
- Een kleine $k$ betekent dat het model sterk reageert op individuele datapunten en kan overfitten.
- Een grotere $k$ maakt het model gladder en helpt bij generalisatie.

Daarnaast zorgt weights = distance ervoor dat buren die dichterbij liggen meer invloed hebben op de voorspelling, wat zorgt voor een stabieler resultaat.

In [5]:
knn_model, knn_params, knn_f1 = train_knn(X_scaled, y)

KeyboardInterrupt: 

## 3.2 Logistic Regression

### Model 2: Logistic Regression

Hoe het model werkt:
Logistische regressie voorspelt de kans dat een observatie tot klasse 1 behoort met behulp van de sigmoidfunctie:

$$
P(y=1 | x) = \frac{1}{1 + e^{-(w^T x + b)}}
$$

Als deze kans groter is dan 0.5 voorspelt het model klasse 1, anders klasse 0.

Loss-functie:
Om te meten hoe goed de voorspellingen zijn wordt de log-loss gebruikt, ook bekend als binaire cross-entropy:

$$
L(y, \hat{y}) = - [y \log(\hat{y}) + (1 - y) \log(1 - \hat{y})]
$$

Gradient Descent:
Het model past de gewichten aan door de loss te minimaliseren met gradient descent:

$$
w_{\text{new}} = w_{\text{old}} - \alpha \frac{\partial L}{\partial w}
$$

Hierbij is α de learning rate. Bijvoorbeeld, als de gradient 4 is en α 0.1, wordt w aangepast met 0.4 richting het minimum van de loss.

Regularisatie:
Regularisatie helpt overfitting te voorkomen door grote coëfficiënten te straffen.

- L1 (Lasso) kan sommige gewichten nul maken en werkt zo ook als feature selectie.
- L2 (Ridge) maakt gewichten kleiner zonder ze nul te maken, waardoor het model beter generaliseert.

Beste hyperparameters:
C = 100, penalty = l2, solver = liblinear.
Met deze instellingen behaalt het model een F1-score van 0.105.

Regularisatie en overfitting:
Hoge gewichten laten zien dat het model sterk afhankelijk is van bepaalde features, wat overfitting kan veroorzaken. Door regularisatie worden de gewichten verlaagd, wat helpt om het model beter te laten generaliseren.

In [6]:
lr_model, lr_params, lr_f1 = train_lr(X_scaled, y)

Best Params: {'lr__C': 100, 'lr__penalty': 'l2', 'lr__solver': 'liblinear'}
Best F1 score: 0.10512810274749802


## 3.3 SVM

### Model 3: Support Vector Machine (SVM)

Hoe het model werkt:
SVM is een classificatiemodel dat een optimale scheidingslijn, of hypervlak, zoekt om de twee klassen van elkaar te scheiden met de grootst mogelijke marge:

$$
\text{maximize } \frac{2}{||w||} \text{ onder de constraint } y_i (w^T x_i + b) \ge 1
$$

Hierbij zijn:
- \(w\) de gewichten, \(b\) de bias
- \(x_i\) de featurevector van observatie \(i\)
- \(y_i \in \{-1, 1\}\) de klasse van observatie \(i\)

Kernel en Kernel Trick:
Wanneer data niet lineair scheidbaar zijn, kan een kernel \(K(x_i, x_j)\) de data transformeren naar een hogere dimensie:

$$
K(x_i, x_j) = \phi(x_i)^T \phi(x_j)
$$

De kernel trick maakt het mogelijk \(\phi(x)\) niet expliciet te berekenen, waardoor het model efficiënter blijft. Bijvoorbeeld, met twee features \((x_1, x_2)\) en een linear kernel:

$$
K(x_i, x_j) = x_i^T x_j = x_{i1} x_{j1} + x_{i2} x_{j2}
$$

Loss-functie:
SVM gebruikt hinge loss:

$$
L(y, \hat{y}) = \max(0, 1 - y \hat{y})
$$

Met \(\hat{y} = w^T x + b\). Het verlies is nul als een voorbeeld correct is geclassificeerd met marge groter dan of gelijk aan 1, anders neemt het lineair toe.

Regularisatie:
De C-parameter reguleert de balans tussen correcte classificatie en maximale marge.
- Hoger C betekent minder foutacceptatie
- Lager C laat een grotere marge toe

De totale loss met regularisatie kan worden weergegeven als:

$$
\min_w \frac{1}{2} ||w||^2 + C \sum_i \max(0, 1 - y_i (w^T x_i + b))
$$

Beste hyperparameters:
- C = 0.1
- kernel = linear
- gamma = scale

Met deze instellingen behaalt het model een F1-score van 0.1035.

Gradient / Optimalisatie:
SVM optimaliseert de gewichten \(w\) via convex optimization en gradient-based methodes, zodat de som van de hinge loss en de regularisatieterm minimaal wordt.

In [7]:
svm_model, svm_params, svm_f1 = train_svm(X_scaled, y)

Best Params: {'svm__C': 0.1, 'svm__degree': 2, 'svm__gamma': 'scale', 'svm__kernel': 'linear'}
Best F1 score: 0.10351940940691495


## 3.4 Decision Tree

### Model 4: Decision Tree

Hoe het model werkt:
Decision Tree splitst de data op basis van beslissingen over features. Elke splitsing verdeelt de data in twee takken totdat een stopcriterium is bereikt, zoals een maximale diepte of minimaal aantal voorbeelden in een blad.

Splitsingscriteria:
De splitsing wordt gekozen op basis van het criterium dat de impurity of onzekerheid het meest vermindert. Twee veelgebruikte criteria zijn:

- Gini impurity:

$$
Gini = 1 - \sum_{i=1}^{C} p_i^2
$$

- Entropy of Information Gain:

$$
Entropy = - \sum_{i=1}^{C} p_i \log_2(p_i)
$$

Hierbij is \(p_i\) de kans dat een voorbeeld tot klasse \(i\) behoort.

Regularisatie en Pruning:
Om overfitting te voorkomen kunnen verschillende parameters worden gebruikt:
- max_depth: maximale boomdiepte
- min_samples_split: minimaal aantal voorbeelden nodig om een knoop te splitsen
- min_samples_leaf: minimaal aantal voorbeelden per blad
- Pruning: takken die weinig bijdragen worden verwijderd

Pruning vermindert overfitting, maar te veel pruning kan juist leiden tot underfitting.

Beste hyperparameters:
- criterion: gini
- max_depth: 5
- min_samples_split: 2
- min_samples_leaf: 4

Met deze instellingen behaalt het model een F1-score van 0.0854.

In [8]:
dt_model, dt_params, dt_f1 = train_decision_tree(X_scaled, y)

Best Params: {'dt__criterion': 'gini', 'dt__max_depth': 5, 'dt__min_samples_leaf': 4, 'dt__min_samples_split': 2}
Best F1 score: 0.0854327793718217


## 3.5 Ensembles (Random Forest, Gradient Boosted Boosted Decistion Trees en XGboost)

### Ensembles: Random Forest, Gradient Boosted Trees en XGBoost

Random Forest (RF):
Random Forest combineert meerdere beslisbomen via bagging, oftewel bootstrap aggregation. Elke boom wordt onafhankelijk getraind, waardoor training relatief snel kan verlopen en parallel uitgevoerd wordt. Belangrijke hyperparameters zijn n_estimators, max_depth, min_samples_split, min_samples_leaf en criterion. De beste instellingen voor dit model zijn criterion = gini, max_depth = 10, min_samples_leaf = 2, min_samples_split = 2 en n_estimators = 100. Met deze instellingen behaalt het model een F1-score van 0.0975.

Gradient Boosted Trees (GBT):
GBT traint meerdere bomen achter elkaar met boosting. Elke nieuwe boom corrigeert de fouten van de vorige bomen. Dit geeft vaak betere prestaties, maar de training duurt langer omdat het sequential is. Belangrijke hyperparameters zijn n_estimators, learning_rate, max_depth, min_samples_split en min_samples_leaf. De beste instellingen zijn learning_rate = 0.1, max_depth = 3, min_samples_leaf = 2, min_samples_split = 2 en n_estimators = 100. Dit levert een F1-score van 0.1011.

XGBoost:
XGBoost is een geavanceerde vorm van boosting, die door optimalisaties zoals parallel split search sneller kan trainen. Extra hyperparameters zoals subsample en colsample_bytree helpen bij regularisatie en verminderen variance. Belangrijke hyperparameters zijn n_estimators, learning_rate, max_depth, subsample en colsample_bytree. De beste instellingen zijn colsample_bytree = 0.8, learning_rate = 0.1, max_depth = 3, n_estimators = 100 en subsample = 1.0. Dit model behaalt een F1-score van 0.1016.

In [5]:
rf_model, rf_params, rf_f1 = train_random_forest(X_scaled, y)
gb_model, gb_params, gb_f1 = train_gradient_boosting(X_scaled, y)
xg_model, xg_params, xg_f1 = train_xgboost(X_scaled, y)

KeyboardInterrupt: 

## 3.6 Zelf samengesteld ensemble

### Custom Ensemble Model

Werking van het ensemble:
Het ensemble combineert minimaal drie verschillende modellen: KNN, Logistic Regression en Random Forest. Hierbij wordt soft voting gebruikt, wat betekent dat de voorspelde kansen van de basismodellen gemiddeld worden. De klasse met de hoogste gemiddelde kans wordt gekozen als uiteindelijke voorspelling.

Keuze van modellen:
- KNN: afstand-gebaseerd en gevoelig voor de schaal van features.
- Logistic Regression: lineair en kans-gebaseerd.
- Random Forest: non-lineair, robuust tegen outliers en kan interacties tussen features oppikken.

Beste hyperparameters:
- knn_n_neighbors = 9
- knn_weights = distance
- lr_C = 10
- rf_max_depth = 5
- rf_n_estimators = 50

Met deze instellingen behaalt het ensemble een F1-score van 0.095.

Waarom dit ensemble werkt:
Door verschillende modellen te combineren verminderen we fouten die individuele modellen maken. Dit helpt vooral bij een dataset met veel meer negatieve dan positieve gevallen, omdat sommige modellen beter presteren bij het herkennen van de zeldzame positieve klasse.

In [6]:
custom_model, custom_params, custom_f1 = train_custom_ensemble(X_scaled, y)

KeyboardInterrupt: 

In [16]:
#Export
# - ChatGPT, 2025, prompt 1: Export. https://chatgpt.com/share/68e262a7-8b80-800a-9b55-d7e59d1f731a
test_ids = df_test['id']
predictions = custom_model.predict(X_scaled_test)
df_submission['id'] = test_ids
df_submission['stroke'] = predictions

df_submission.to_csv('data/submission_custom_ensemble.csv', index=False)
#print(df_submission.describe())

test_ids = df_test['id']

trained_models = {
    'knn': knn_model,
    'logreg': lr_model,
    'svm': svm_model,
    'decision_tree': dt_model,
    'random_forest': rf_model,
    'gradient_boosting': gb_model,
    'xgboost': xg_model,
    'custom_ensemble': custom_model
}

for name, model in trained_models.items():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f'data/submission_{name}_{timestamp}.csv'
    export_submission(model, X_scaled_test, test_ids, filename)

                 id       stroke
count   8388.000000  8388.000000
mean   37094.643777     0.101335
std    20955.196907     0.301790
min        4.000000     0.000000
25%    18877.750000     0.000000
50%    37490.000000     0.000000
75%    55290.250000     0.000000
max    72934.000000     1.000000
Submission saved as data/submission_knn_20251005_141957.csv
Submission saved as data/submission_logreg_20251005_141957.csv
Submission saved as data/submission_svm_20251005_141957.csv
Submission saved as data/submission_decision_tree_20251005_142000.csv
Submission saved as data/submission_random_forest_20251005_142000.csv
Submission saved as data/submission_gradient_boosting_20251005_142000.csv
Submission saved as data/submission_xgboost_20251005_142000.csv
Submission saved as data/submission_custom_ensemble_20251005_142000.csv


# 4.0 Referentielijst
- scikit-learn, 2025, make_scorer: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html
- Imbalanced learning, 2025, SMOTE: https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTE.html
- scikit-learn, 2025, StarfieldKFold: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
- ChatGPT, 2025, prompt 1/2: Markdown Table Help. https://chatgpt.com/share/68e0c97c-687c-8002-b0f8-10e5e1e77ccb
- ChatGPT, 2025, prompt 1: Export. https://chatgpt.com/share/68e262a7-8b80-800a-9b55-d7e59d1f731a
- ChatGPT, 2025, prompt: Verbeteren van custom ensemble model: https://chatgpt.com/share/68e281a6-b384-800a-8744-210bd4b3c038